In [ ]:
# importing the neccesary packages
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from sklearn.model_selection import train_test_split
import keras
import numpy as np
import pandas as pd 
import os
import cv2
import matplotlib.pyplot as plt

In [ ]:
data_path = os.listdir("H:/UK/Ann/train")


In [ ]:
sample_img = plt.imread("H:/UK/Ann/train/yawn/10.jpg")

In [ ]:
def face_yawn(direc="H:/UK/Ann/train", fac_path="H:/UK/Ann/haarcascade_frontalface_default.xml"):
    yaw_no = []
    IMG_SIZE = 145
    categories = ["yawn", "no_yawn"]
    for category in categories:
        link_path = os.path.join(direc, category)
        class_num1 = categories.index(category)
        print(class_num1)
        for image in os.listdir(link_path):
            image_array = cv2.imread(os.path.join(link_path, image), cv2.IMREAD_COLOR)
            face_cascade = cv2.CascadeClassifier(fac_path)
            faces = face_cascade.detectMultiScale(image_array, 1.3, 5)
            for (x, y, w, h) in faces:
                img = cv2.rectangle(image_array, (x, y), (x+w, y+h), (0, 255, 0), 2)
                roi_color = img[y:y+h, x:x+w]
                resized_array = cv2.resize(roi_color, (IMG_SIZE, IMG_SIZE))
                yaw_no.append([resized_array, class_num1])
    return yaw_no


yawn_no_yawn = face_yawn()

In [ ]:
def get_data(dir_path="H:/UK/Ann/train", face_cas="H:/UK/Ann/haarcascade_frontalface_default.xml", eye_cas="H:/UK/Ann/haarcascade.xml"):
    data_path = ['Closed', 'Open']
    IMG_SIZE = 145
    data = []
    for label in data_path:
        path = os.path.join(dir_path, label)
        class_num = data_path.index(label)
        class_num +=2
        print(class_num)
        for img in os.listdir(path):
            try:
                img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_COLOR)
                resized_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                data.append([resized_array, class_num])
            except Exception as e:
                print(e)
    return data

In [ ]:
data_train = get_data()


In [ ]:
def append_data():
    yaw_no = face_yawn()
    data = get_data()
    yaw_no.extend(data)
    return np.array(yaw_no)

In [ ]:
new_data = append_data()


In [ ]:
X = []
y = []
for feature, label in new_data:
    X.append(feature)
    y.append(label)

In [ ]:
X = np.array(X)
X = X.reshape(-1, 145, 145, 3)

In [ ]:
labeling = LabelBinarizer()
y = labeling.fit_transform(y)

In [ ]:
y = np.array(y)

In [ ]:
seed = 42
test_size = 0.30
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=seed, test_size=test_size)

In [ ]:
train_gen = ImageDataGenerator(rescale=1/255, zoom_range=0.2, horizontal_flip=True, rotation_range=30)
test_gen = ImageDataGenerator(rescale=1/255)

train_gen = train_gen.flow(np.array(X_train), y_train, shuffle=False)
test_gen = test_gen.flow(np.array(X_test), y_test, shuffle=False)

In [ ]:
model_def = Sequential()

model_def.add(Conv2D(256, (3, 3), activation="relu", input_shape=X_train.shape[1:]))
model_def.add(MaxPooling2D(2, 2))

model_def.add(Conv2D(128, (3, 3), activation="relu"))
model_def.add(MaxPooling2D(2, 2))

model_def.add(Conv2D(64, (3, 3), activation="relu"))
model_def.add(MaxPooling2D(2, 2))

model_def.add(Conv2D(32, (3, 3), activation="relu"))
model_def.add(MaxPooling2D(2, 2))

model_def.add(Flatten())
model_def.add(Dropout(0.5))

model_def.add(Dense(64, activation="relu"))
model_def.add(Dense(4, activation="softmax"))

model_def.compile(loss="categorical_crossentropy", metrics=["accuracy"], optimizer="adam")

model_def.summary()

In [ ]:
History = model_def.fit(train_gen, epochs=25, validation_data=test_gen, shuffle=True, validation_steps=len(test_gen))

In [ ]:
accuracy = History.History ['accuracy']
val_accuracy = History.History ['val_accuracy']
loss = History.History['loss']
val_loss = History.History['val_loss']
epochs = range(len(accuracy))

plt.plot(epochs, accuracy, "g", label="trainning accuracy")
plt.plot(epochs, val_accuracy, "r", label="validation accuracy")
plt.legend()
plt.show()

plt.plot(epochs, loss, "g", label="trainning loss")
plt.plot(epochs, val_loss, "r", label="validation loss")
plt.legend()
plt.show()

In [ ]:
model_def.save("dorwsii.h5")


In [ ]:
prediction=np.argmax(model_def.predict(X_test), axis=-1)

In [ ]:
labels_new = ["yawn", "no_yawn", "Closed", "Open"]
IMG_SIZE = 145
def predict(filepath, face_cas="H:/UK/Ann/haarcascade_frontalface_default.xml"):
    img_array = cv2.imread(filepath, cv2.IMREAD_COLOR)
    img_array = img_array / 255
    resized_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    return resized_array.reshape(-1, IMG_SIZE, IMG_SIZE, 3)

model_pre = tf.keras.models.load_model("C:/Users/91915/dorwsii.h5")

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(np.argmax(y_test, axis=1), prediction, target_names=labels_new))

In [ ]:
prediction = model_pre.predict([predict("H:/UK/Ann/testing/test5cl.jpg")])
np.argmax(prediction)